## Import Necessary Packages

In [1]:
import glob
import pickle
import numpy as np
import xarray as xr
import proplot as pplt
%config InlineBackend.figure_format = 'retina'
pplt.rc.update({'font.size':12})
import warnings
warnings.filterwarnings('ignore')

## Import Data

We load in the variables necessary to calculate the terms within the $\langle h_f \rangle'^2$ budget equation (Eq. 3). Where ```lwns``` and ```lwnt``` are net longwave at the surface and top of atmopshere, respectively, ```swns``` and ```swnt``` are net shortwave at the surface and top of atmopshere, respectively, ```lhf``` is latent heat flux, ```shf``` is sensible heat flux, and ```hf``` is the column integral of frozen moist static energy ($\langle h_f \rangle$).

In [2]:
path  = '/ocean/projects/atm200007p/ajenney/rcemip-large/'
cases = ['nz_32','nz_64','nz_128']
hf    = []
lwnt  = []
lwns  = []
swnt  = []
swns  = []
shf   = []
lhf   = []
for i,case in enumerate(cases):
    print('Working on '+case)
    hf.append(xr.open_mfdataset(np.sort(glob.glob(path+case+'/OUT_3D/FMSE/*.nc'))).fmse_vint)
    lwnt.append(xr.open_mfdataset(np.sort(glob.glob(path+case+'/OUT_2D/LWNT/*.nc'))).LWNT)
    lwns.append(xr.open_mfdataset(np.sort(glob.glob(path+case+'/OUT_2D/LWNS/*.nc'))).LWNS)
    swnt.append(xr.open_mfdataset(np.sort(glob.glob(path+case+'/OUT_2D/SWNT/*.nc'))).SWNT)
    swns.append(xr.open_mfdataset(np.sort(glob.glob(path+case+'/OUT_2D/SWNS/*.nc'))).SWNS)
    shf.append(xr.open_mfdataset(np.sort(glob.glob(path+case+'/OUT_2D/SHF/*.nc'))).SHF)
    lhf.append(xr.open_mfdataset(np.sort(glob.glob(path+case+'/OUT_2D/LHF/*.nc'))).LHF)

Working on nz_32
Working on nz_64
Working on nz_128


## Calculate Eq. 3 $\langle h_f \rangle'^2$ Budget Terms

We calculate the longwave convergence ($LW$), shortwave convergence ($SW$), and surface enthalpy flux ($SEF$) from the above variables, and calculate daily averages of each (including $h_f$).

In [3]:
sw  = []
lw  = []
sef = []
lwdaily  = []
swdaily  = []
sefdaily = []
hfdaily  = []
for i,case in enumerate(cases):
    print('Working on '+case)
    ## Calculate LW, SW, and SEF
    lw.append(lwns[i]-lwnt[i])
    sw.append(swnt[i]-swns[i])
    sef.append(lhf[i]+shf[i])
    ## Compute Daily Means of LW, SW, SEF, and FMSE
    lwdaily.append(lw[i].groupby(np.floor(lw[i].time)).mean('time').load())
    swdaily.append(sw[i].groupby(np.floor(sw[i].time)).mean('time').load())
    sefdaily.append(sef[i].groupby(np.floor(lw[i].time)).mean('time').load())
    hfdaily.append(hf[i].groupby(np.floor(hf[i].time)).mean('time').load())

Working on nz_32
Working on nz_64
Working on nz_128


We calculate the spatial anomalies for daily-averaged variables as the deviation from the daily-averaged mean. We use $LW'$, $SW'$, $SEF'$, and $h_{f}'$), to compute the spatial means of individual terms within Eq. 3. Expanding Eq. 3 gives us,
$$\frac{1}{2}\frac{\partial \langle h_f \rangle'^2}{\partial t} = \langle h_f \rangle' LW' + \langle h_f \rangle' SW' + \langle h_f \rangle' SEF' - \langle h_f \rangle' \left(\nabla_h \cdot \langle h_f\vec{\textbf{u}} \rangle' \right)$$
Represented in the following code as ```totaltend``` = ```lwterm``` + ```swterm``` + ```sefterm``` - ```fluxconv```.


In [18]:
lwanom  = []
swanom  = []
sefanom = []
hfanom  = []
lwterm  = []
swterm  = []
sefterm = []
totaltend = []
fluxconv  = []
for i,case in enumerate(cases):
    print('Working on '+case)
    ## Compute Spatial Anomalies of LW, SW, SEF, and FMSE
    lwanom.append(lwdaily[i]-(lwdaily[i].mean(dim=('x','y'))))
    swanom.append(swdaily[i]-(swdaily[i].mean(dim=('x','y'))))
    sefanom.append(sefdaily[i]-(sefdaily[i].mean(dim=('x','y'))))
    hfanom.append(hfdaily[i]-(hfdaily[i].mean(dim=('x','y'))))
    ## Calculate Individual Terms and Spatial Means
    lwterm.append((lwanom[i]*hfanom[i]).mean(dim=('x','y'))/1e9)
    swterm.append((swanom[i]*hfanom[i]).mean(dim=('x','y'))/1e9)
    sefterm.append((sefanom[i]*hfanom[i]).mean(dim=('x','y'))/1e9)
    totaltend.append((((1/2)*((hfanom[i]**2).differentiate('time'))).mean(dim=('x','y')))/86400/1e9)
    fluxconv.append((totaltend[i]-lwterm[i]-swterm[i]-sefterm[i]))

Working on nz_32
Working on nz_64
Working on nz_128


## Store Budget Terms by Case Using Pickling

We use pickling to format and store the ```totaltend```, ```lwterm```, ```swterm```, ```sefterm```, and ```fluxconv``` terms by case for quicker plotting of Fig. 6 and Fig. 8. Note that pickling is only used for **short-term storage**. More information on pickling can be found in the Python 3.10.7 documentation [here](https://docs.python.org/3/library/pickle.html).

In [ ]:
### Create Arrays of FMSE Budget Terms
swarr = xr.DataArray(swterm,name='SW',dims=('cases','days')).assign_attrs(
    long_name='Convergence of shortwave radiative flux between the surface and TOA',
    units='10$^{9}$ J$^{2}$ m$^{-4}$ s$^{-1}$')
lwarr = xr.DataArray(lwterm,name='LW',dims=('cases','days')).assign_attrs(
    long_name='Convergence of longwave radiative flux between the surface and TOA',
    units='10$^{9}$ J$^{2}$ m$^{-4}$ s$^{-1}$')
sefarr = xr.DataArray(sefterm,name='SEF',dims=('cases','days')).assign_attrs(
    long_name='Surface enthalpy (latent + sensible heat) flux',
    units='10$^{9}$ J$^{2}$ m$^{-4}$ s$^{-1}$')
fluxconvarr = xr.DataArray(fluxconv,name='flux_conv',dims=('cases','days')).assign_attrs(
    long_name='Horizontal convergence of column frozen moist static energy flux',
    units='10$^{9}$ J$^{2}$ m$^{-4}$ s$^{-1}$',
    description='$-\nabla_h \cdot \widehat{\overrightarrow{u}h_f}$')
totaltendarr = xr.DataArray(totaltend,name='total_tend',dims=('cases','days')).assign_attrs(
    long_name='Total tendency of column frozen moist static energy',
    units='10$^{9}$ J$^{2}$ m$^{-4}$ s$^{-1}$',
    description='$(\frac{1}{2} \frac{\partial \widehat{h_f}^2}{\partial t}))$')

### Merge Arrays By Case
nz_32_vars  = xr.merge([swarr[0],lwarr[0],sefarr[0],fluxconvarr[0],totaltendarr[0]])
nz_64_vars  = xr.merge([swarr[1],lwarr[1],sefarr[1],fluxconvarr[1],totaltendarr[1]])
nz_128_vars = xr.merge([swarr[2],lwarr[2],sefarr[2],fluxconvarr[2],totaltendarr[2]])

### Define a Function to Pickle Arrays and Save as Files to a Desired Directory
def picklebycase(data,filename):
    filepath = path+'analysis/'+filename
    outfile  = open(filepath,'wb')
    pickle.dump(data,outfile)
    outfile.close()
    
### Pickle Case Arrays
picklebycase(nz_32_vars,'fmse_budget_vars_nz_32.pkl')
picklebycase(nz_64_vars,'fmse_budget_vars_nz_64.pkl')
picklebycase(nz_128_vars,'fmse_budget_vars_nz_128.pkl')